# Проект по определению токсичных комментариев (выполнен в google colaboratory)

## Введение

В данном проекте будут построены модели для поиска токсичных комментариев в описании товаров вики-сообщества интернет-магазина. Для каждой модели будет рассчитана метрика качества (F1). Данная метрика используется, т.к. в данной задаче для нас очень корректно дифференцировать именно токсичные комментарии.

## Подготовка

Установим и загрузим необходимые библиотеки.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 31.8MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 3.3MB 40.5MB/s 


In [2]:
!pip install catboost

     |████████████████████████████████| 69.2MB 42kB/s 


In [3]:
import torch
import transformers as ppb
import pandas as pd
import numpy as np
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=3,suppress=True)
import warnings
warnings.simplefilter("ignore")
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
import time
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
m = Mystem() 
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from lightgbm import LGBMClassifier

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
data = pd.read_csv('C:/Users/Lantana/Documents/data_science/13_text/toxic_comments.csv')

In [5]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Проведём анализ сбалансированности выборки.

In [6]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [7]:
data['toxic'].value_counts(normalize=True)

0   0.898
1   0.102
Name: toxic, dtype: float64

Разделим данные на обучающую и тестовую выборки.

In [8]:
train, test = train_test_split(data, test_size=0.25, random_state=12345)

In [9]:
train['toxic'].value_counts()

0    107540
1     12138
Name: toxic, dtype: int64

Выборка несбалансированна - токсичных комментариев в ней всего 10%. Выполним уменьшение обучающей выборки для последующей корректной работы моделей, тестовую выборку оставим без изменений. Создадим функцию downsample, которая разделяет выборку на отрицательные и положительные объекты, случайным образом отбрасывает часть объектов отрицательного класса и перемешивает данные. Затем применим её к нашим данным.

In [10]:
target_for_downsampling = train['toxic']

In [11]:
def downsample(data, target, fraction):
    data_zeros = data[target == 0]
    data_ones = data[target == 1]

    data_downsampled = pd.concat(
        [data_zeros.sample(frac=fraction, random_state=12345)] + [data_ones])
    
    data_downsampled = shuffle(data_downsampled, random_state=12345)
    
    return data_downsampled

train_downsampled = downsample(train, target_for_downsampling, 0.2)

In [12]:
train_downsampled['toxic'].value_counts()

0    21508
1    12138
Name: toxic, dtype: int64

In [13]:
train_downsampled['toxic'].value_counts(normalize=True)

0   0.639
1   0.361
Name: toxic, dtype: float64

In [14]:
train = train_downsampled.reset_index(drop=True)

In [15]:
test = test.reset_index(drop=True)

In [16]:
test['toxic'].value_counts()

0    35806
1     4087
Name: toxic, dtype: int64

In [17]:
test['toxic'].value_counts(normalize=True)

0   0.898
1   0.102
Name: toxic, dtype: float64

Очистим данные в столбце text от лишних символов с помощью регулярных выражений. Дополнительно уберём лишние пробелы.

In [18]:
len(train)

33646

In [19]:
len(test)

39893

In [20]:
train_clean = []
for i in range(len(train)):
    text = re.sub(r'[^a-zA-Z ]', ' ', train['text'][i])
    text = text.split() 
    clear = " ".join(text)
    clear = clear.lower()
    train_clean.append(clear)

In [21]:
train['text'] = train_clean

In [22]:
train.head()

,text,toxic
0,dude what the hell is with all these incorrect...,1
1,you stupid ass fucker retarded whore incompete...,1
2,removal of three names from roster list i have...,0
3,and offer you sexual favours,1
4,welcome hello and welcome to wikipedia thank y...,0


In [23]:
test_clean = []
for i in range(len(test)):
    text2 = re.sub(r'[^a-zA-Z ]', ' ', test['text'][i])
    text2 = text2.split() 
    clear2 = " ".join(text2)
    clear2 = clear2.lower()
    test_clean.append(clear2)

In [24]:
test['text'] = test_clean

In [25]:
test.head()

,text,toxic
0,ahh shut the fuck up you douchebag sand nigger...,1
1,reply there is no such thing as texas commerce...,0
2,reply hey you could at least mention jasenovac...,0
3,thats fine there is no deadline chi,0
4,dyk nomination of mustarabim hello your submis...,0


## <font color='blue'>Векторизация данных</font>

In [26]:
train_vector = train

In [27]:
test_vector = test

<font color='blue'>Лемматизируем train и test</font>

In [28]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = lemmatizer.lemmatize(text)
    lemm_text = "".join(lemm_list)
    return lemm_text

In [29]:
train_vector['lemm'] = train_vector['text'].apply(lemmatize)

In [30]:
train_vector.head()

,text,toxic,lemm
0,dude what the hell is with all these incorrect...,1,dude what the hell is with all these incorrect...
1,you stupid ass fucker retarded whore incompete...,1,you stupid ass fucker retarded whore incompete...
2,removal of three names from roster list i have...,0,removal of three names from roster list i have...
3,and offer you sexual favours,1,and offer you sexual favours
4,welcome hello and welcome to wikipedia thank y...,0,welcome hello and welcome to wikipedia thank y...


In [31]:
test_vector['lemm'] = test_vector['text'].apply(lemmatize)

In [32]:
test_vector.head()

,text,toxic,lemm
0,ahh shut the fuck up you douchebag sand nigger...,1,ahh shut the fuck up you douchebag sand nigger...
1,reply there is no such thing as texas commerce...,0,reply there is no such thing as texas commerce...
2,reply hey you could at least mention jasenovac...,0,reply hey you could at least mention jasenovac...
3,thats fine there is no deadline chi,0,thats fine there is no deadline chi
4,dyk nomination of mustarabim hello your submis...,0,dyk nomination of mustarabim hello your submis...


<font color='blue'>Лемматизируем train и test</font>

In [33]:
features_train_vector = train_vector['lemm'].values.astype('U')
target_train_vector = train_vector['toxic']
features_test_vector = test_vector['lemm'].values.astype('U')
target_test_vector = test_vector['toxic']

<font color='blue'>Вычислим TF-IDF для корпуса текстов. Cоздадим счётчик count_tf_idf, указав в нём стоп-слова. </font>

In [34]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train_vector)
tf_idf_test = count_tf_idf.transform(features_test_vector)
                                                

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [35]:
display(tf_idf_train.shape)
display(tf_idf_test.shape)

(33646, 65924)

(39893, 65924)

<font color='blue'>Применим модели логистической регрессии, решающего дерева и LGBMClassifier для классификации тональности текста, взяв TF-IDF как признаки. </font>

### <font color='blue'>Модель логистической регрессиии (подход с векторизацией) </font>

In [36]:
model_vector = LogisticRegression(random_state=12345, class_weight='balanced')
model_vector.fit(tf_idf_train, target_train_vector)
predictions_test_vector = model_vector.predict(tf_idf_test)

In [37]:
result_test_vector = f1_score(target_test_vector, predictions_test_vector)
print("Значение F1-меры модели логистической регрессии на тестовой выборке методом векторизации:", result_test_vector)

Значение F1-меры модели логистической регрессии на тестовой выборке методом векторизации: 0.705375050140393


### <font color='blue'>Модель решающего дерева (подход с векторизацией) </font>

In [38]:
for depth in range(1, 11):
    model_tree_vector = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    model_tree_vector.fit(tf_idf_train, target_train_vector)
    predictions_tree_vector = model_tree_vector.predict(tf_idf_test) 
    print("max_depth =", depth, ": ", end='')
    result_tree_vector = f1_score(target_test_vector, predictions_tree_vector)
    print("F1:", result_tree_vector)

max_depth = 1 : F1: 0.276006711409396
max_depth = 2 : F1: 0.37364371670941016
max_depth = 3 : F1: 0.3734701934465061
max_depth = 4 : F1: 0.4258943781942078
max_depth = 5 : F1: 0.42993809791783905
max_depth = 6 : F1: 0.46950890447922283
max_depth = 7 : F1: 0.5004356159609689
max_depth = 8 : F1: 0.5278013943206937
max_depth = 9 : F1: 0.5483709273182957
max_depth = 10 : F1: 0.5427399507793272


### <font color='blue'>Модель LGBMClassifier (подход с векторизацией) </font>

In [39]:
model_light = LGBMClassifier(random_state=12345, n_estimators=100)
model_light.fit(tf_idf_train, target_train_vector, verbose=10)
predictions_light = model_light.predict(tf_idf_test)
result_light = f1_score(target_test_vector, predictions_light)
print("Значение F1-меры модели LGBMClassifier на тестовой выборке", result_light)

Значение F1-меры модели LGBMClassifier на тестовой выборке 0.750996737948532


## <font color='blue'>Модель BERT </font>

Загрузим предобученную BERT-модель

Для анализа возьмём выборку из<font color='blue'> обучающих данных в размере 1500 строк и из тестовых данных в размере 500 строк (тогда сохраняются пропорции тестовой выборки = 25% от всех данных). </font>Больше не позволят мощности компьютера и google colaboratory. Но данных строк уже достаточно для создания качественной модели.

In [40]:
train = train.sample(1500).reset_index(drop=True)

In [41]:
test = test.sample(500).reset_index(drop=True)

In [42]:
train['toxic'].value_counts(normalize=True)

0   0.633
1   0.367
Name: toxic, dtype: float64

In [43]:
test['toxic'].value_counts(normalize=True)

0   0.920
1   0.080
Name: toxic, dtype: float64

In [44]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токенизируем данные, применим padding к векторам и создадим маску для выделения важных токенов.

In [45]:
tokenized_train = train['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

In [46]:
tokenized_test = test['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

In [47]:
max_len = 0
for i in tokenized_train.values:
    if len(i) > max_len:
        max_len = len(i)

padded_train = np.array([i + [0]*(max_len - len(i)) for i in tokenized_train.values])
np.array(padded_train).shape

(1500, 512)

In [48]:
max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len - len(i)) for i in tokenized_test.values])
np.array(padded_test).shape

(500, 512)

In [49]:
attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_train.shape

(1500, 512)

In [50]:
attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

(500, 512)

In [51]:
batch_size = 100
embeddings_train = []
for i in notebook.tqdm(range(padded_train.shape[0] // batch_size)):
        batch = torch.LongTensor(padded_train[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask_train[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings_train.append(batch_embeddings[0][:,0,:].numpy())

In [52]:
batch_size = 100
embeddings_test = []
for i in notebook.tqdm(range(padded_test.shape[0] // batch_size)):
        batch2 = torch.LongTensor(padded_test[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch2 = torch.LongTensor(attention_mask_test[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings2 = model(batch2, attention_mask=attention_mask_batch2)
        
        embeddings_test.append(batch_embeddings2[0][:,0,:].numpy())

И извлечём признаки для обучения: целевой - target, остальные - features.

In [53]:
features_train = np.concatenate(embeddings_train)
target_train = train['toxic']
features_test = np.concatenate(embeddings_test)
target_test = test['toxic'] 

In [54]:
display(len(features_train))
display(len(target_train))
display(len(features_test))
display(len(target_test))

1500

1500

500

500

**Вывод**

Было произведено уменьшение выборки в части отрицательных классов для повышения точности работы моделей в дальнейшем. Данные были токенизированы, к ним был применён padding и были созданы маски для выделения важных токенов. Затем данные были разбиты на обучающую и тестовую выборки, извлечены признаки для обучения. В работе анализируется сбалансированная по классам в целевом признаке выборка в размере 5000 строк.

## Обучение

Рассмотрим модели логистической регрессии, решающего дерева, случайного леса и Catboost. Подберём для них наилучшие гиперпараметры.

### Модель логистической регрессии

In [55]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(features_train, target_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 78.94738947368421}
best scrores:  0.8426666666666668


In [62]:
model = LogisticRegression(C=78.94738947368421, random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)

LogisticRegression(C=78.94738947368421, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
predictions_test = model.predict(features_test)
result_test = f1_score(target_test, predictions_test)
print("Значение F1-меры модели логистической регрессии на тестовой выборке:", result_test)

Значение F1-меры модели логистической регрессии на тестовой выборке: 0.43609022556390975


### Модель решающего дерева

In [58]:
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_test) 
    print("max_depth =", depth, ": ", end='')
    result_tree = f1_score(target_test, predictions_tree)
    print("F1:", result_tree)

max_depth = 1 : F1: 0.2617801047120419
max_depth = 2 : F1: 0.2608695652173913
max_depth = 3 : F1: 0.24074074074074078
max_depth = 4 : F1: 0.24870466321243523
max_depth = 5 : F1: 0.2545454545454546
max_depth = 6 : F1: 0.308641975308642
max_depth = 7 : F1: 0.3006535947712418
max_depth = 8 : F1: 0.3037974683544304
max_depth = 9 : F1: 0.3
max_depth = 10 : F1: 0.3116883116883116


Для модели решающего дерева наилучшее значение метрики F1 = 0,31 получилось при максимальной глубине = 10

### Модель случайного леса

In [59]:
best_model_forest = None
best_result_forest = 0
best_est_forest = 0
best_depth_forest = 0
for est in range(10, 101, 10):
    for depth_forest in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth_forest, class_weight='balanced')
        model_forest.fit(features_train, target_train)
        predictions_forest = model_forest.predict(features_test)
        result_forest = f1_score(target_test, predictions_forest)
        if result_forest > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result_forest
            best_est_forest = est
            best_depth_forest = depth_forest


print("F1 наилучшей модели случайного леса на тестовой выборке:", best_result_forest, "Количество деревьев:", best_est_forest, "Глубина:", best_depth_forest)

F1 наилучшей модели случайного леса на тестовой выборке: 0.5979381443298969 Количество деревьев: 100 Глубина: 8


### Модель Catboost

In [60]:
model_cat = CatBoostClassifier(loss_function="Logloss", random_seed=12345, iterations=150)
model_cat.fit(features_train, target_train, verbose=10)
predictions_cat = model_cat.predict(features_test)
result_cat = f1_score(target_test, predictions_cat)
print("Значение F1-меры модели Catboost на тестовой выборке:", result_cat)


Learning rate set to 0.069767
0:	learn: 0.6590837	total: 324ms	remaining: 48.3s
10:	learn: 0.4377189	total: 2.24s	remaining: 28.3s
20:	learn: 0.3383702	total: 4.13s	remaining: 25.4s
30:	learn: 0.2874526	total: 6s	remaining: 23.1s
40:	learn: 0.2474739	total: 7.86s	remaining: 20.9s
50:	learn: 0.2173829	total: 9.7s	remaining: 18.8s
60:	learn: 0.1961289	total: 11.7s	remaining: 17s
70:	learn: 0.1734945	total: 13.5s	remaining: 15s
80:	learn: 0.1581259	total: 15.3s	remaining: 13.1s
90:	learn: 0.1416354	total: 17.2s	remaining: 11.1s
100:	learn: 0.1288925	total: 19s	remaining: 9.23s
110:	learn: 0.1158649	total: 20.9s	remaining: 7.33s
120:	learn: 0.1009716	total: 22.7s	remaining: 5.44s
130:	learn: 0.0896112	total: 24.5s	remaining: 3.56s
140:	learn: 0.0796549	total: 26.4s	remaining: 1.68s
149:	learn: 0.0702434	total: 28s	remaining: 0us
Значение F1-меры модели Catboost на тестовой выборке: 0.5436893203883495


## Выводы

В данном проекте были построены модели для поиска токсичных комментариев в описании товаров вики-сообщества интернет-магазина. Были рассмотрены модели логистической регрессии, решающего дерева, случайного леса и Catboost. Для каждой модели была рассчитана метрика качества (F1).

Первоначально было произведено уменьшение выборки в части отрицательных классов для повышения точности работы моделей в дальнейшем. Данные были токенизированы, к ним был применён padding и были созданы маски для выделения важных токенов. В работе анализируется сбалансированная по классам в целевом признаке выборка в размере 5000 строк.

Выведем итоговую таблицу с результатами для каждой рассмотренной модели:

In [61]:
summary = pd.DataFrame({'model': ['LogisticRegression с векторизацией', 'DecisionTreeClassifier с векторизацией', 'LGBMClassifier с векторизацией', 'LogisticRegression (BERT)', 'DecisionTreeClassifier (BERT)', 'RandomForestClassifier (BERT)', 'CatBoostClassifier (BERT)'], 'F1': [result_test_vector, result_tree_vector, result_light, result_test, result_tree, result_forest, result_cat]})
summary

,model,F1
0,LogisticRegression с векторизацией,0.705
1,DecisionTreeClassifier с векторизацией,0.543
2,LGBMClassifier с векторизацией,0.751
3,LogisticRegression (BERT),0.450
4,DecisionTreeClassifier (BERT),0.312
5,RandomForestClassifier (BERT),0.547
6,CatBoostClassifier (BERT),0.544


Лидером оказалась модель LGBMClassifier с векторизацией со значением метрики f1 = 0.751